7.1处理缺失数据

pandas使用浮点值NAN表示缺失数据，称其为哨兵值，可以很方便的检测出来

In [1]:
import pandas as pd
import numpy as np
string_data=pd.Series(['aardvark','artichoke',np.nan,'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [2]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

过滤缺失数据大方法有很多，可以通过pandas.isnull或手工索引的方法，但dropna可能会更实用一点。对于一个Series,dropna返回一个仅含非空数据和索引值的Series

In [3]:
from numpy import nan as NA
data=pd.Series([1,NA,3.5,NA,7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

这等价于

In [4]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

对于DataFrame你可能希望丢弃NA或含有NA的行货列，drop默认丢弃含有缺失值的行

In [5]:
data=pd.DataFrame([[1.,6.5,3.],[1.,NA,NA],
                  [NA,NA,NA],[NA,6.5,3.]])
clean=data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [6]:
clean

,0,1,2
0,1.0,6.5,3.0


传入how='all'将只丢弃全为NA的行

In [7]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


用这种方式丢弃列，只需axis=1

In [8]:
data[4]=NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [9]:
data.dropna(axis=1,how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [10]:
df=pd.DataFrame(np.random.randn(7,3))
df.iloc[:4,1]=NA
df.iloc[:2,2]=NA
df

,0,1,2
0,0.981373,NaN,NaN
1,1.548466,NaN,NaN
2,0.474190,NaN,-0.999463
3,0.904751,NaN,-0.199000
4,0.317943,-0.497954,0.181159
5,-0.710899,0.987922,0.530774
6,1.076164,0.542332,-0.930149


滤除DataFrame行的问题涉及时间序列数据，假设你只想留下一部分观测数据，thresh参数实现此目的

In [11]:
df.dropna(thresh=2)

,0,1,2
2,0.474190,NaN,-0.999463
3,0.904751,NaN,-0.199000
4,0.317943,-0.497954,0.181159
5,-0.710899,0.987922,0.530774
6,1.076164,0.542332,-0.930149


填充缺失数据，

In [12]:
df.fillna(0)

,0,1,2
0,0.981373,0.000000,0.000000
1,1.548466,0.000000,0.000000
2,0.474190,0.000000,-0.999463
3,0.904751,0.000000,-0.199000
4,0.317943,-0.497954,0.181159
5,-0.710899,0.987922,0.530774
6,1.076164,0.542332,-0.930149


In [13]:
df.fillna({1:0.5,2:0})

,0,1,2
0,0.981373,0.500000,0.000000
1,1.548466,0.500000,0.000000
2,0.474190,0.500000,-0.999463
3,0.904751,0.500000,-0.199000
4,0.317943,-0.497954,0.181159
5,-0.710899,0.987922,0.530774
6,1.076164,0.542332,-0.930149


fillna默认返回新对象，但也可以对现有对象进行修改

In [14]:
#_=df.fillna(0,inplace=True)
#df

In [15]:
df.fillna(method='ffill')

,0,1,2
0,0.981373,NaN,NaN
1,1.548466,NaN,NaN
2,0.474190,NaN,-0.999463
3,0.904751,NaN,-0.199000
4,0.317943,-0.497954,0.181159
5,-0.710899,0.987922,0.530774
6,1.076164,0.542332,-0.930149


用fillna实现许多其他别的功能，，

In [16]:
data=pd.Series([1.,NA,3.5,NA,7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

7.2数据转换

移除重复数据

In [17]:
data=pd.DataFrame({'k1':['one','two']*3+['two'],
                   'k2':[1,1,2,3,3,4,4]})
                 
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


duplivated返回一个布尔型Series,表示各行是否重复行

In [18]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [19]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


这两个方法默认判断全部列，也可以指定部分列，只希望根据k1过滤

In [20]:
data['v1']=range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


传入keep=last则保留最后一个

In [21]:
data.drop_duplicates(['k1','k2'],keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


利用函数或映射进行数据转换
对于很多数据集，你可能希望根据数组，Series,DataFrame列中的之实现转化

In [22]:
import pandas as pd
data=pd.DataFrame({'food':['bocan','pulled pork','bocan',
                          'Pastrami','corned beef','Bocan',
                          'pastrami','honey ham','nova lox'],
                  'ounces':[4,3,12,6,7.5,8,3,5,6]})
data

,food,ounces
0,bocan,4.0
1,pulled pork,3.0
2,bocan,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bocan,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


再添加一列该肉类食物来源的动物类型

In [23]:
meat_to_animal={
    'bocan':'pig',
    'pulled pork':'pig',
    'pastrami':'cow',
    'corned beef':'cow',
    'honey ham':'pig',
    'nova lox':'salimon'
    
}

有些肉类的首字母大写，还需要使用Series的str.lower方法，转换为小写

In [24]:
lowercased=data['food'].str.lower()
lowercased

0          bocan
1    pulled pork
2          bocan
3       pastrami
4    corned beef
5          bocan
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [25]:
data['animal']=lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bocan,4.0,pig
1,pulled pork,3.0,pig
2,bocan,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bocan,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salimon


我们也可以传入一个能完成全部这些工作的函数

In [26]:
data['food'].map(lambda x:meat_to_animal[x.lower()])

0        pig
1        pig
2        pig
3        cow
4        cow
5        pig
6        cow
7        pig
8    salimon
Name: food, dtype: object

替换值

In [27]:
data=pd.Series([1.,-999.,2.,-999,-1000.,-3])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5      -3.0
dtype: float64

-999坑是是一个缺失数据的标记值，要将其替换为pandas能够理解的NA值

In [28]:
import numpy as np
data.replace(-999,np.nan)


0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5      -3.0
dtype: float64

In [29]:
data.replace([-999,-1000],np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5   -3.0
dtype: float64

In [30]:
data.replace([-999,-1000],[np.nan,0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5   -3.0
dtype: float64

重命名轴索引

In [31]:
data=pd.DataFrame(np.arange(12).reshape((3,4)),
                 index=['Ohio','colordo','new york'],
                 columns=['one','two','three','four'])
data

,one,two,three,four
Ohio,0,1,2,3
colordo,4,5,6,7
new york,8,9,10,11


跟Series一样，轴索引也有map方法

In [32]:
transform=lambda x:x[:4].upper()
data.index.map(transform)


Index(['OHIO', 'COLO', 'NEW '], dtype='object')

可以将其赋值给index，就可以对DataFrame就地修改

In [33]:
data.index=data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


如果想要创建数据集的转化版，而不是修改原始数据，使用的方法是rename

In [34]:
data.rename(index=str.title,columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [35]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


rename可以结合字典型对象实现对轴标签的更新

In [36]:
data.rename(index={'OHIO':'INDIANA'},inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


离散化和面元划分
假设一组人员数据，，而你希望将它们划分成不同的年龄组

In [37]:
ages=[20,22,25,27,21,23,37,31,61,45,41,32]


接下来将这些数据划分到18到25，26~35，等几个面元

In [38]:
bins=[18,25,35,60,100]
cats=pd.cut(ages,bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

pandas返回的是一个特殊面元的Categories对象，结果展示了pandas.cut划分的面元，你可以将其看作一组表示面元名称的字符串

In [39]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [40]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [41]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

跟区间的符号一样，哪边是闭端可以通过right=False进行修改

In [42]:
pd.cut(ages,[18,26,36,61,100],right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

可以通过传递一个列表或数组到labels,设置自己的面元名称

In [43]:
group_names=['young','youngadult','middlenaged','senior']
pd.cut(ages,bins,labels=group_names)

[young, young, young, youngadult, young, ..., youngadult, senior, middlenaged, middlenaged, youngadult]
Length: 12
Categories (4, object): [young < youngadult < middlenaged < senior]

如果向cut传入面元的数量而不是确切边界，则它会根据数据的最大最小值计算等长面元

In [44]:
data=np.random.rand(20)
pd.cut(data,4,precision=2)

[(0.0047, 0.2], (0.0047, 0.2], (0.59, 0.78], (0.2, 0.39], (0.0047, 0.2], ..., (0.0047, 0.2], (0.39, 0.59], (0.0047, 0.2], (0.39, 0.59], (0.39, 0.59]]
Length: 20
Categories (4, interval[float64]): [(0.0047, 0.2] < (0.2, 0.39] < (0.39, 0.59] < (0.59, 0.78]]

qcut是一个非常类似于cut的函数，它可以根据样本分位数对数据进行面元划分

In [45]:
data=np.random.rand(1000)
cats=pd.qcut(data,4)
cats

[(0.499, 0.756], (0.256, 0.499], (0.756, 0.998], (0.256, 0.499], (-0.000761, 0.256], ..., (0.756, 0.998], (-0.000761, 0.256], (0.499, 0.756], (0.256, 0.499], (0.756, 0.998]]
Length: 1000
Categories (4, interval[float64]): [(-0.000761, 0.256] < (0.256, 0.499] < (0.499, 0.756] < (0.756, 0.998]]

In [46]:
pd.value_counts(cats)

(0.756, 0.998]        250
(0.499, 0.756]        250
(0.256, 0.499]        250
(-0.000761, 0.256]    250
dtype: int64

检测和过滤异常值

过滤和变换异常值（outlier）在很大程度上就是运用数组运算

In [47]:
data=pd.DataFrame(np.random.randn(1000,4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.021511,0.046273,-0.013427,-0.016542
std,1.050727,0.990563,0.958633,0.964081
min,-2.921386,-3.418942,-3.306707,-2.651841
25%,-0.728859,-0.622994,-0.672099,-0.650160
50%,0.018780,0.032633,-0.039418,-0.019120
75%,0.681927,0.734571,0.598220,0.621656
max,3.401472,3.572370,3.575383,2.992538


想找找某列中绝对值大小超过3的值

In [48]:
col=data[2]
col[np.abs(col)>3]

508    3.575383
628   -3.306707
699   -3.132159
Name: 2, dtype: float64

要选出全部超过3，-3的值，可以在布尔型dATAFrame中使用any

In [49]:
data[(np.abs(data)>3).any(1)]

,0,1,2,3
118,-0.581510,-3.418942,2.024317,-0.460408
181,3.401472,-1.584162,-0.144716,-1.681711
401,0.072659,3.572370,1.954941,0.023291
419,3.266082,-1.596525,-0.601242,-0.337181
508,-1.296683,-0.780237,3.575383,-1.684058
628,3.079951,0.425292,-3.306707,-1.753170
690,-0.183090,3.517775,-0.147588,0.373570
699,0.431720,1.218592,-3.132159,0.531279


In [50]:
data[np.abs(data)>3]=np.sign(data)*3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.022258,0.045602,-0.013564,-0.016542
std,1.048459,0.985617,0.955216,0.964081
min,-2.921386,-3.000000,-3.000000,-2.651841
25%,-0.728859,-0.622994,-0.672099,-0.650160
50%,0.018780,0.032633,-0.039418,-0.019120
75%,0.681927,0.734571,0.598220,0.621656
max,3.000000,3.000000,3.000000,2.992538


上面的代码可以将值限制在-3到3之间，根据数据的值是正是负，np.sign(data)可以生成1，-1

In [51]:
np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,1.0,-1.0
1,-1.0,-1.0,1.0,-1.0
2,1.0,-1.0,1.0,1.0
3,1.0,-1.0,1.0,-1.0
4,1.0,1.0,-1.0,-1.0


排列和随机采样
利用numpy.random.permutation(permuting,随机重排序)，可以轻松实现对Series,DataFRAME列的排列工作

In [52]:
df=pd.DataFrame(np.arange(5*4).reshape((5,4)))
sampler=np.random.permutation(5)
sampler

array([3, 1, 2, 4, 0])

In [53]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


按行随机排列

In [54]:
df.take(sampler)

,0,1,2,3
3,12,13,14,15
1,4,5,6,7
2,8,9,10,11
4,16,17,18,19
0,0,1,2,3


In [57]:
mnames=['movie_id','title','genres']
movies=pd.read_table('D:/download/pydata-book-2nd-edition/datasets/movielens/movies.dat',sep='::',
                    header=None,names=mnames)
movies[:10]

D:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [59]:
all_genres=[]
for x in movies.genres:
    all_genres.extend(x.split('|'))
    genres=pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [60]:
zero_matrix=np.zeros((len(movies),len(genres)))
dummies=pd.DataFrame(zero_matrix,columns=genres)
gen=movies.genres[0]
gen.split('|')


['Animation', "Children's", 'Comedy']

In [61]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

In [63]:
for i,gen in enumerate(movies.genres):
    indices=dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i,indices]=1

In [66]:
movies_windic=movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[2]

movie_id                                   3
title                Grumpier Old Men (1995)
genres                        Comedy|Romance
Genre_Animation                            0
Genre_Children's                           0
Genre_Comedy                               1
Genre_Adventure                            0
Genre_Fantasy                              0
Genre_Romance                              1
Genre_Drama                                0
Genre_Action                               0
Genre_Crime                                0
Genre_Thriller                             0
Genre_Horror                               0
Genre_Sci-Fi                               0
Genre_Documentary                          0
Genre_War                                  0
Genre_Musical                              0
Genre_Mystery                              0
Genre_Film-Noir                            0
Genre_Western                              0
Name: 2, dtype: object

第八章

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [3]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [4]:
data['b']
data['b':'c']
data.loc[['b', 'd']]

1   -0.555730
3    1.965781
dtype: float64

b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
dtype: float64

b  1   -0.555730
   3    1.965781
d  2    0.281746
   3    0.769023
dtype: float64

In [5]:
data.loc[:, 2]

a    0.478943
c    0.092908
d    0.281746
dtype: float64

In [6]:
data.unstack()

,1,2,3
a,-0.204708,0.478943,-0.519439
b,-0.555730,NaN,1.965781
c,1.393406,0.092908,NaN
d,NaN,0.281746,0.769023


In [7]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

使用DataFrame的列进行索引
df.set_index()
接受单个字符或者是字符列表
drop=True 默认设为索引后该列被删
df.reset_index()
将索引变成一列

In [8]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [9]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1